# Demo: Bench4M for classification Task

In [1]:
from benchmol.trainer.trainer_linear_probing_fusion import (
    FusionModelConfig, OptimConfig,
    create_trainer_from_paths, get_default_task_setup
)

/home/ubuntu/anaconda3/envs/benchmol/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ==== user configs ====
dataset = "sider"
csv_path = f"./Bench3M-4M-features-MoleculeNet/{dataset}/{dataset}_processed_ac.csv"
feat_pkl_paths = [
    f"./Bench3M-4M-features-MoleculeNet/{dataset}/fingerprints@mcfp4_2048.pkl",
    f"./Bench3M-4M-features-MoleculeNet/{dataset}/geometry@unimol_10conf.pkl",
    f"./Bench3M-4M-features-MoleculeNet/{dataset}/geometry_image@IEM_3d_10conf.pkl",
    f"./Bench3M-4M-features-MoleculeNet/{dataset}/graph@MoleBERT.pkl"
]
task_type = "classification"  # or "regression"
split_column = "scaffold_split"  # train/valid/test

device = "cuda:0"  # or "cpu"
runseed = 0
batch_size = 32
lr = 0.001
weights = [0.6, 0.1, 0.1, 0.2]
num_workers = 2

# Fusion: maps each input to common_dim then fuses with opt
fusion_cfg = FusionModelConfig(
    arch="arch3",
    common_dim=-1,
    hidden_dim_list=None,     # e.g., [512, 256]
    opt="weighted_sum",       # sum/mean/max/min/sub/concat/weighted_sum
    activation="linear",
    weights=weights,          # e.g., [0.2,0.3,0.5] when opt='weighted_sum'
)

optim_cfg = OptimConfig(optim="adam", lr=lr)

In [3]:
# Build trainer (loads CSV+PKLs, builds fusion model + optimizer)
trainer, (idx_train, idx_valid, idx_test) = create_trainer_from_paths(
    csv_path=csv_path,
    feat_pkl_paths=feat_pkl_paths,
    dataset=dataset,
    task_type=task_type,
    split_column=split_column,
    batch_size=batch_size,
    num_workers=num_workers,
    device=device,
    runseed=runseed,
    fusion_cfg=fusion_cfg,
    optim_cfg=optim_cfg,
)

print(trainer.model)

BaseFusionLayer(
  (fc_layers): ModuleList(
    (0): Sequential(
      (linear1): Linear(in_features=2048, out_features=2048, bias=True)
      (activator1): Identity()
    )
    (1): Sequential(
      (linear1): Linear(in_features=512, out_features=2048, bias=True)
      (activator1): Identity()
    )
    (2): Sequential(
      (linear1): Linear(in_features=512, out_features=2048, bias=True)
      (activator1): Identity()
    )
    (3): Sequential(
      (linear1): Linear(in_features=300, out_features=2048, bias=True)
      (activator1): Identity()
    )
  )
  (activation): Identity()
  (task_layer): Sequential(
    (linear): Linear(in_features=2048, out_features=27, bias=True)
  )
)


In [4]:
# Train
num_epochs = 100
eval_metric, valid_select, min_value, _, _ = get_default_task_setup(dataset, task_type)

results = trainer.train(
    num_epochs=num_epochs,
    eval_metric=eval_metric,
    valid_select=valid_select,
    min_value=min_value,
    dataset=dataset,
    save_dir="./experiments/Bench4M/linear_probing_fusion",
    save_data_prediction_dict=True,
    save_finetune_ckpt=True,
    idx_splits=(idx_train, idx_valid, idx_test),
)
results

[train] dataset: sider; epoch: 0 loss: 0.537: 100%|██████████| 36/36 [00:01<00:00, 22.66it/s]


[train] dataset: sider; epoch: 0 loss: 0.537


[eval on train set] dataset: sider; epoch: 0 loss: 0.512: 100%|██████████| 36/36 [00:00<00:00, 180.18it/s]
[eval on valid set] dataset: sider; epoch: 0 loss: 0.486: 100%|██████████| 5/5 [00:00<00:00, 39.40it/s]
[eval on test set] dataset: sider; epoch: 0 loss: 0.500: 100%|██████████| 5/5 [00:00<00:00, 37.80it/s]


{'dataset': 'sider', 'epoch': 0, 'loss': 0.5120494630601671, 'metric': 'ROCAUC', 'train': 0.6794269696818626, 'valid': 0.5599277258433694, 'test': 0.5991626069321043}
model has been saved as ./experiments/Bench4M/linear_probing_fusion/valid_best.pth


[train] dataset: sider; epoch: 1 loss: 0.500: 100%|██████████| 36/36 [00:00<00:00, 108.71it/s]


...

[train] dataset: sider; epoch: 99 loss: 0.017: 100%|██████████| 36/36 [00:00<00:00, 112.84it/s]


[train] dataset: sider; epoch: 99 loss: 0.017


[eval on train set] dataset: sider; epoch: 99 loss: 0.019: 100%|██████████| 36/36 [00:00<00:00, 160.98it/s]
[eval on valid set] dataset: sider; epoch: 99 loss: 1.497: 100%|██████████| 5/5 [00:00<00:00, 32.96it/s]
[eval on test set] dataset: sider; epoch: 99 loss: 1.938: 100%|██████████| 5/5 [00:00<00:00, 34.59it/s]


{'dataset': 'sider', 'epoch': 99, 'loss': 0.018507278627819486, 'metric': 'ROCAUC', 'train': 0.999977133653728, 'valid': 0.6224236098812762, 'test': 0.5814573375103899}
final results: {'highest_valid': 0.6408292903017184, 'final_train': 0.9239096063106433, 'final_test': 0.6350642960850951, 'highest_train': 0.9999905752994198}


{'highest_valid': 0.6408292903017184,
 'final_train': 0.9239096063106433,
 'final_test': 0.6350642960850951,
 'highest_train': 0.9999905752994198,
 'highest_valid_desc': defaultdict(float,
             {'accuracy': 0.7847707847707848,
              'ROCAUC': 0.6408292903017184,
              'f1': 0.6549115427392923,
              'AUPR': 0.6960982962801298,
              'precision': 0.7079200642246157,
              'recall': 0.7331020393727417,
              'kappa': 0.08162944401301918,
              'matthews': 0.113485158047291,
              'result_list_dict_each_task': [{'accuracy': 0.6573426573426573,
                'ROCAUC': 0.7143987341772153,
                'f1': 0.751269035532995,
                'AUPR': 0.7114043040547442,
                'precision': 0.6271186440677966,
                'recall': 0.9367088607594937,
                'kappa': 0.2645113886847906,
                'matthews': 0.3262541198165554,
                'fpr': array([0.      , 0.      , 0.015625, 0.